In [57]:
import read_write_model as colmap_reader
import numpy as np
import torch
import imageio.v3 as iio
import matplotlib.pyplot as plt
from load_data import read_data

In [58]:
imagedir = './images'
camFile = 'cameras.bin'
imgFile = 'images.bin'

images, c2w, H, W, F = read_data(camFile, imgFile, imagedir)

Loading Poses...
Done
Reading Images...
Done


In [59]:
i, j = torch.meshgrid(torch.arange(0, W, dtype = torch.int32), torch.arange(0, H, dtype=torch.int32), indexing = 'xy')
dirs = torch.stack([(i - W * 0.5) / F, -(j - H * .5)/F, -torch.ones_like(i)], -1)
rays_d = torch.sum(dirs[..., np.newaxis, :] * c2w[0, :3, :3], -1)
rays_o = torch.broadcast_to(torch.Tensor(c2w[0, :3, -1]), rays_d.shape)

In [75]:
L_embed = 10
def posenc(x):
  rets = [x]
  for i in range(L_embed):
    for fn in [torch.sin, torch.cos]:
      rets.append(fn(2.**i * x))
      print(fn(2.**i * x).shape)
      return
  return torch.concat(rets, -1)

In [76]:
rays_o.shape

torch.Size([2304, 3072, 3])

In [77]:
test = posenc(rays_o)

torch.Size([2304, 3072, 3])


In [69]:
test.shape

torch.Size([2304, 3072, 63])

In [ ]:
class PositionalEncoding():
    def __init__(self, L_pos = 10, L_dir = 4):
        '''
        L_pos: Position (xyz) coordinates will be mapped to L_pos * 2 dimentions
        L_dir: Viewing direction unit vector will be mapped to L_dir * 2 dimentions
        '''
        self.L_pos = 10
        self.L_dir = 4

    def forward(pos, vdir):
        posvec = torch.zeros(pos.shape[0], L_pos * 2 * 3)
        dirvec = torch.zeros(vdir.shape[0], L_pos * 2 * 3)

        
        